In [5]:
# Install packages
#import sys
#!{sys.executable} -m pip install gensim

In [6]:
# Import libraries
import pandas as pd
#from google.colab import files
import io
import string
import nltk
#import logging
#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level = logging.ERROR)
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
import random
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.style.use('dark_background')
from operator import itemgetter
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from sklearn.manifold import TSNE
from project_lib import Project

In [11]:
#nltk.download('all')

In [8]:
# Init
project = Project(project_id = 'cdd4213e-9372-44a8-b9ed-d2e5f051bd4d', project_access_token = 'p-bbc42375d46b8499812dda0574c7dea672c617ae')
pc = project.project_context

# Fetch 
f = project.get_file("vaccination_all_tweets.csv")

# Read the CSV data file into a pandas DataFrame
f.seek(0)
df = pd.read_csv(f)
df.head(n = 10)

,id,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,retweets,favorites,is_retweet
0,1340539111971516416,Rachel Roh,"La Crescenta-Montrose, CA",Aggregator of Asian American news; scanning di...,2009-04-08 17:52:46,405,1692,3247,False,2020-12-20 06:06:44,Same folks said daikon paste could treat a cyt...,['PfizerBioNTech'],Twitter for Android,0,0,False
1,1338158543359250433,Albert Fong,"San Francisco, CA","Marketing dude, tech geek, heavy metal & '80s ...",2009-09-21 15:27:30,834,666,178,False,2020-12-13 16:27:13,While the world has been on the wrong side of ...,NaN,Twitter Web App,1,1,False
2,1337858199140118533,eli🇱🇹🇪🇺👌,Your Bed,"heil, hydra 🖐☺",2020-06-25 23:30:28,10,88,155,False,2020-12-12 20:33:45,#coronavirus #SputnikV #AstraZeneca #PfizerBio...,"['coronavirus', 'SputnikV', 'AstraZeneca', 'Pf...",Twitter for Android,0,0,False
3,1337855739918835717,Charles Adler,"Vancouver, BC - Canada","Hosting ""CharlesAdlerTonight"" Global News Radi...",2008-09-10 11:28:53,49165,3933,21853,True,2020-12-12 20:23:59,"Facts are immutable, Senator, even when you're...",NaN,Twitter Web App,446,2129,False
4,1337854064604966912,Citizen News Channel,NaN,Citizen News Channel bringing you an alternati...,2020-04-23 17:58:42,152,580,1473,False,2020-12-12 20:17:19,Explain to me again why we need a vaccine @Bor...,"['whereareallthesickpeople', 'PfizerBioNTech']",Twitter for iPhone,0,0,False
5,1337852648389832708,Dee,"Birmingham, England","Gastroenterology trainee, Clinical Research Fe...",2020-01-26 21:43:12,105,108,106,False,2020-12-12 20:11:42,Does anyone have any useful advice/guidance fo...,NaN,Twitter for iPhone,0,0,False
6,1337851215875608579,Gunther Fehlinger,"Austria, Ukraine and Kosovo",End North Stream 2 now - the pipeline of corru...,2013-06-10 17:49:22,2731,5001,69344,False,2020-12-12 20:06:00,it is a bit sad to claim the fame for success ...,['vaccination'],Twitter Web App,0,4,False
7,1337850832256176136,Dr.Krutika Kuppalli,NaN,"ID, Global Health, VHF, Pandemic Prep, Emergin...",2019-03-25 04:14:29,21924,593,7815,True,2020-12-12 20:04:29,There have not been many bright days in 2020 b...,"['BidenHarris', 'Election2020']",Twitter for iPhone,2,22,False
8,1337850023531347969,Erin Despas,NaN,Designing&selling on Teespring. Like 90s Disne...,2009-10-30 17:53:54,887,1515,9639,False,2020-12-12 20:01:16,Covid vaccine; You getting it?\n\n #CovidVacci...,"['CovidVaccine', 'covid19', 'PfizerBioNTech', ...",Twitter Web App,2,1,False
9,1337842295857623042,Ch.Amjad Ali,Islamabad,#ProudPakistani #LovePakArmy #PMIK @insafiansp...,2012-11-12 04:18:12,671,2368,20469,False,2020-12-12 19:30:33,#CovidVaccine \n\nStates will start getting #C...,"['CovidVaccine', 'COVID19Vaccine', 'US', 'paku...",Twitter Web App,0,0,False


In [12]:
words = set(nltk.corpus.words.words())
stop_words = stopwords.words('english')

# Function to process text
def process_txt(x):

    x = x.translate(str.maketrans('', '', string.punctuation)).lower()
    x = ''.join([i for i in x if not i.isdigit()]) 
    x = [i for i in x.split(' ') if i in words and len(i) > 2 and i not in stop_words]

    return x   

# Function to filter pos
def filter_pos(x):

    x = nltk.pos_tag(x)
    x = [i[0] for i in x if i[1].startswith(('N', 'A', 'J'))]

    return x

In [13]:
print(f'Dataset Shape:      {df.shape}')
print('Missing Observations- ')
print(df.isnull().sum())

Dataset Shape:      (37880, 16)
Missing Observations- 
id                     0
user_name              0
user_location       8631
user_description    2574
user_created           0
user_followers         0
user_friends           0
user_favourites        0
user_verified          0
date                   0
text                   0
hashtags            8072
source                31
retweets               0
favorites              0
is_retweet             0
dtype: int64


In [14]:
# Apply functions to process text
df['txt'] = df['text'].apply(process_txt)

# Filter empty
df1 = df[df.astype(str)['txt'] != "[]"]

# Apply function to filter POS
df1['pos'] = df1['txt'].apply(filter_pos)

# Filter tweets that have been deleted
df2 = df1[df1.astype(str)['pos'] != "[]"]

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [15]:
print(f'{len(df) - len(df1)} tweets were filtered out due to processing..')
print(f'\n{len(df1) - len(df2)} observations dropped due to grammar..')

797 tweets were filtered out due to processing..

992 observations dropped due to grammar..


In [16]:
df2['pos'][0:10]

0                                [daikon, paste, storm]
1     [world, wrong, side, history, year, biggest, v...
2                                [covid, vaccine, last]
3                    [immutable, senator, sturdy, born]
5        [anyone, useful, covid, vaccine, safe, whilst]
6     [bit, sad, claim, fame, success, vaccination, ...
7                                  [many, bright, days]
8                               [covid, vaccine, covid]
9                                               [start]
10                    [mark, millions, people, vaccine]
Name: pos, dtype: object

In [17]:
# Create Dictionary
words = corpora.Dictionary(df2['pos'])

# Term Document Frequency
corpus = [words.doc2bow(text) for text in df2['pos']]

In [21]:
# Iterate to find optimal topics
#scores = []

#for n in range(1, 25, 1):

#    model = gensim.models.ldamodel.LdaModel(corpus = corpus,
#                                            id2word = words,
#                                            num_topics = n, 
#                                            random_state = 100,
#                                            update_every = 1,
#                                            chunksize = 100,
#                                            passes = 10,
#                                            alpha = 'auto',
#                                            per_word_topics = True)
    
    # Score (low = good)
#    c = CoherenceModel(model = model, texts = df2['pos'], dictionary = words, coherence = 'c_v')
#    coherence = c.get_coherence()
#    perplexity = model.log_perplexity(corpus)

#    print(f'LDA for {n} topics, Coherence Score: ', coherence)

#    scores.append((n, coherence, perplexity))

#fig, ax = plt.subplots(figsize = (25, 10))
#plt.title('Coherence Scores for LDA w/ N Topics', fontsize = 12)
#plt.xlabel('# Topics', fontsize = 12)
#plt.ylabel('Coherence', fontsize = 12)
#plt.plot([i[0] for i in scores], [i[1] for i in scores], lw = .5)
#fig.show();

In [28]:
# Build optimal model
model = gensim.models.ldamodel.LdaModel(corpus = corpus,
                                        id2word = words,
                                        num_topics = 8, 
                                        random_state = 100,
                                        update_every = 1,
                                        chunksize = 100,
                                        passes = 10,
                                        alpha = 'auto',)
                                        #per_word_topics = True)

In [29]:
for index, topic in model.show_topics(formatted = False, num_words = 10):

    print('Topic: {} \nWords: {}'.format(index, ', '.join([w[0] for w in topic])))

Topic: 0 
Words: today, russia, everything, jab, side, effects, price, time, blood, fund
Topic: 1 
Words: business, vaccination, first, country, health, second, minister, update, march, way
Topic: 2 
Words: good, government, news, thanks, supply, last, china, production, company, purchase
Topic: 3 
Words: vaccine, covid, open, new, ford, stay, shot, get, effective, world
Topic: 4 
Words: usual, start, approval, thank, ready, great, president, spring, see, high
Topic: 5 
Words: dose, people, day, trial, happy, possible, want, agreement, next, take
Topic: 6 
Words: use, emergency, yesterday, part, home, best, corona, team, ministry, test
Topic: 7 
Words: canada, batch, arm, days, many, efficacy, safety, order, cost, third


In [30]:
# Convert
topic_assignments = model.get_document_topics(corpus, minimum_probability = 0)
csr = gensim.matutils.corpus2csc(topic_assignments)
topic_array = csr.T.toarray()
topics_df = pd.DataFrame(topic_array)

In [32]:
topics_df.columns = [n for n in range(1, 9)]
topics_df.head()

,1,2,3,4,5,6,7,8
0,0.126931,0.144094,0.076587,0.338966,0.070766,0.095528,0.062221,0.084907
1,0.127818,0.172662,0.055994,0.412833,0.051746,0.069190,0.046204,0.063552
2,0.113376,0.127238,0.107174,0.375412,0.062089,0.083019,0.055438,0.076254
3,0.123537,0.144219,0.073946,0.341831,0.067835,0.095898,0.066111,0.086624
4,0.102575,0.115133,0.060832,0.434636,0.092431,0.075174,0.050209,0.069008


In [33]:
df2['topic'] = topics_df.idxmax(axis = 1)

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [34]:
df2['topic'].value_counts()

4.0    34341
2.0       19
1.0       15
6.0        4
7.0        1
Name: topic, dtype: int64

In [ ]:
# Compute
tfidf = TfidfVectorizer()
x = tfidf.fit_transform(df2['pos'].apply(lambda x: ' '.join(x)))
tsne = TSNE(n_components = 2, random_state = 100).fit_transform(x)

df2['t1'] = tsne[:, 0]
df2['t2'] = tsne[:, 1]

In [ ]:
# Visualize
fig, ax = plt.subplots(figsize = (30, 15))
ax.scatter(df2['t1'], df2['t2'], s = .5, c = df2['topic'])
plt.xlabel('t1', fontsize = 12)
plt.ylabel('t2', fontsize = 12)
plt.title('t-SNE scores for COVID vaccination tweets')